# Youtube

Download data from 
https://www.kaggle.com/datasnaek/youtube-new

And copy it in the `datasets` folder.

Don't unzip
            
    

In [ ]:
    
import zipfile
import pandas as pd
import io
import json

countries = {}
categories = {}

with zipfile.ZipFile("datasets/youtube-new.zip") as z:
    for file in z.filelist:
        print(f'processing {file.filename}')
        key = file.filename.split('.')[0]        
        if file.filename.endswith('.csv'):
            with z.open(file.filename) as f:
                countries[key] = pd.read_csv(io.StringIO(f.read().decode('latin-1')))
        if file.filename.endswith('.json'):                
            with z.open(file.filename) as f:
                categories[key] = json.loads(f.read())

## Let's have a look at our data

In [2]:
df = countries['CAvideos']
df.describe()

,category_id,views,likes,dislikes,comment_count
count,40881.000000,4.088100e+04,4.088100e+04,4.088100e+04,4.088100e+04
mean,20.795553,1.147036e+06,3.958269e+04,2.009195e+03,5.042975e+03
std,6.775054,3.390913e+06,1.326895e+05,1.900837e+04,2.157902e+04
min,1.000000,7.330000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,20.000000,1.439020e+05,2.191000e+03,9.900000e+01,4.170000e+02
50%,24.000000,3.712040e+05,8.780000e+03,3.030000e+02,1.301000e+03
75%,24.000000,9.633020e+05,2.871700e+04,9.500000e+02,3.713000e+03
max,43.000000,1.378431e+08,5.053338e+06,1.602383e+06,1.114800e+06


In [3]:
print('Unique values for each column')
df.nunique()

Unique values for each column


video_id                  24427
trending_date               205
title                     24573
channel_title              5076
category_id                  17
publish_time              23613
tags                      20157
views                     40170
likes                     24676
dislikes                   6241
comment_count             11172
thumbnail_link            24422
comments_disabled             2
ratings_disabled              2
video_error_or_removed        2
description               22345
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40881 entries, 0 to 40880
Data columns (total 16 columns):
video_id                  40881 non-null object
trending_date             40881 non-null object
title                     40881 non-null object
channel_title             40881 non-null object
category_id               40881 non-null int64
publish_time              40881 non-null object
tags                      40881 non-null object
views                     40881 non-null int64
likes                     40881 non-null int64
dislikes                  40881 non-null int64
comment_count             40881 non-null int64
thumbnail_link            40881 non-null object
comments_disabled         40881 non-null bool
ratings_disabled          40881 non-null bool
video_error_or_removed    40881 non-null bool
description               39585 non-null object
dtypes: bool(3), int64(5), object(8)
memory usage: 4.2+ MB


## Objectives
1. What are the top 10 tags for each channel ?
2. What are the top 25 tags in one country ?
3. Fix the bugs !
4. Multiprocessing - What are the top 25 tags in EACH country ?
4. For each of the top 25 tags (worldwide), calculate the ratios: like, dislike & comments (by view)

## 1. What are the top 10 tags for each channel ?

A nice use case for a custom group by

Custom group by may seem a bit complicated at first, but they give a lot of flexibility - this way you don't have to learn the whole pandas library !

We're going to create a custom group by *step by step*, so that you can understand the thought process you have to go through.


In [5]:
# we have to build a custom group by
# let's creae a group by object containing the data of a single entity
# we'll get quick calculations this way

# a group by object performs no calculations in itself

gb = df.loc[df['channel_title'] == '20th Century Fox'].groupby('channel_title')['tags']


In [6]:
# working on a single, but large group lets us experiment quickly while covering most edge cases

# we create a very basic custom function that we use to observe the data we receive from the groupby aggregation
# let's get the inner variable to experiment with it

def foo(x):
    print('start')
    print(x)
    print(type(x))
    print('end')   
    return []

gb.agg(foo)

start
291      Trailer|"Hugh Jackman"|"Michelle Williams"|"Za...
400      Trailer|"Deadpool"|"20th Century Fox (Producti...
600      Trailer|"Deadpool"|"20th Century Fox (Producti...
800      Trailer|"Deadpool"|"20th Century Fox (Producti...
1001     Trailer|"Deadpool"|"20th Century Fox (Producti...
1210     Trailer|"Deadpool"|"20th Century Fox (Producti...
1499     Trailer|"Deadpool"|"20th Century Fox (Producti...
3073     Trailer|"Nick Robinson"|"Katherine Langford"|"...
3234     Trailer|"Nick Robinson"|"Katherine Langford"|"...
3454     Trailer|"Nick Robinson"|"Katherine Langford"|"...
3792     Trailer|"Nick Robinson"|"Katherine Langford"|"...
5018     Trailer|"alita"|"battle angel"|"alita: battle ...
5203     Trailer|"alita"|"battle angel"|"alita: battle ...
5409     Trailer|"alita"|"battle angel"|"alita: battle ...
5641     Trailer|"alita"|"battle angel"|"alita: battle ...
5932     Trailer|"alita"|"battle angel"|"alita: battle ...
11208    Trailer|"Jennifer Lawrence"|"Joel Edgerto

channel_title
20th Century Fox    []
Name: tags, dtype: object

In [7]:
# we start working on the logox
# first we notice that the tags are separated by pipes ('|')
# let's split each of the string by the pipe character
# the "x" object will now containt a list of strings in each row 

def foo(x):
    print('start')
    # split by '|'
    res = x.apply(lambda x: x.split('|'))
    print(res)    
    return 'v'

gb.agg(foo)

start
291      [Trailer, "Hugh Jackman", "Michelle Williams",...
400      [Trailer, "Deadpool", "20th Century Fox (Produ...
600      [Trailer, "Deadpool", "20th Century Fox (Produ...
800      [Trailer, "Deadpool", "20th Century Fox (Produ...
1001     [Trailer, "Deadpool", "20th Century Fox (Produ...
1210     [Trailer, "Deadpool", "20th Century Fox (Produ...
1499     [Trailer, "Deadpool", "20th Century Fox (Produ...
3073     [Trailer, "Nick Robinson", "Katherine Langford...
3234     [Trailer, "Nick Robinson", "Katherine Langford...
3454     [Trailer, "Nick Robinson", "Katherine Langford...
3792     [Trailer, "Nick Robinson", "Katherine Langford...
5018     [Trailer, "alita", "battle angel", "alita: bat...
5203     [Trailer, "alita", "battle angel", "alita: bat...
5409     [Trailer, "alita", "battle angel", "alita: bat...
5641     [Trailer, "alita", "battle angel", "alita: bat...
5932     [Trailer, "alita", "battle angel", "alita: bat...
11208    [Trailer, "Jennifer Lawrence", "Joel Edge

channel_title
20th Century Fox    v
Name: tags, dtype: object

In [18]:
# next we want to combine our lists of strings in a single list
# we use a simple nested loop for this

def foo(x):
    print('start')
    # split by '|'
    res = x.apply(lambda x: x.split('|'))
    
    # flatten in single list
    flat = []
    for row in res:
        for elem in row:
            flat.append(elem.strip('"')) # removes the aesthetically unpleasant "
    print(flat)
    
    # return a set (so it keeps only unique elements)
    return tuple(set(flat))

gb.agg(foo)

start
['Trailer', 'Hugh Jackman', 'Michelle Williams', 'Zac Efron', 'Zendaya', 'Rebecca Ferguson', 'pasek and paul', 'la la land', 'moulin rouge', 'high school musical', 'hugh jackman musical', 'zac efron musical', 'musical', 'the greatest showman', 'greatest showman', 'Michael Gracey', 'P.T. Barnum', 'Barnum and Bailey', 'Barnum Circus', 'Barnum and Bailey Circus', '20th century fox', 'greatest showman trailer', 'trailer', 'official trailer', 'the greatest showman trailer', 'logan', 'Benj Pasek', 'Justin Paul', 'Trailer', 'Deadpool', '20th Century Fox (Production Company)', 'Deadpool Movie', 'Ryan Reynolds (Celebrity)', 'Ed Skrein (Musical Artist)', 'T. J. Miller (TV Writer)', 'Gina Carano (Martial Artist)', 'Red band', 'Red band deadpool', 'Marvel', 'Marvel Comics', 'Comic Book (Comic Book Genre)', 'Dead pool', 'Deadpool green band', 'Deadpool red band', 'Action', 'Comedy', 'Action Comedy', 'X-Men (Award-Winning Work)', 'Trailer', 'Deadpool', '20th Century Fox (Production Company)', 

channel_title
20th Century Fox    (hunger games, Carrie Coon, Jason Matthews, Si...
Name: tags, dtype: object

In [19]:
# let's give a proper name to our function and apply it to the whole dataset
# and remove all the print statements !! else it will freeze our notebook
# The nested loop can be simplified to a list comp

# now we have all unique tags per channel - nice, but not what we need
gb_all = df.groupby('channel_title')[['tags']]

def unique_tags(x):
    # split by '|'
    res = x.apply(lambda x: x.split('|')) 
    # flatten in single list
    flat = [elem.strip('"') for row in res for elem in row]
    # return a set (so it keeps only unique elements)
    return tuple(set(flat))

(gb_all.agg([unique_tags, 'count'])
       .droplevel(level=0, axis='columns')
       .sort_values(by='count', ascending=False)
)

,unique_tags,count
channel_title,,
VikatanTV,"(deiva magal serial 01.12.17, priyamanaval 101...",284
etvteluguindia,"(comedy skits in telugu, latest telugu serials...",282
Flowers Comedy,"(moosa, Juhi Rustagi, college campus comedy, p...",270
ETV Plus India,"(comedy skits in telugu, chalaki chanti, Mahes...",253
SAB TV,"(jetha babita, canada visit, break door, top e...",244
RadaanMedia,"(Radhika in funny character, vellai thamarai, ...",243
mallemalatv,"(, Auto Punches, sekhar master, Sudheer, Rajiv...",238
SET India,"(game show, the kapil sharma show clips, docto...",238
ETV Jabardasth,"(comedy skits in telugu, chalaki chanti, raisi...",234


In [20]:
# we'll use a Counter to get the top 10

from collections import Counter

gb_all = df.groupby('channel_title')[['tags']]

def unique_tags(x):
    # split by '|'
    res = x.apply(lambda x: x.split('|')) 
    # flatten in single list
    flat = [elem.strip('"') for row in res for elem in row]
    # return a set (so it keeps only unique elements)
    return Counter(flat).most_common(10)

(gb_all.agg([unique_tags, 'count'])
       .droplevel(level=0, axis='columns')
       .sort_values(by='count', ascending=False)
)

,unique_tags,count
channel_title,,
VikatanTV,"[(priyamanaval serial, 169), (piriyamanaval, 1...",284
etvteluguindia,"[(jabardasth, 271), (extra jabardasth, 264), (...",282
Flowers Comedy,"[(flowers tv, 268), (flower tv comedy, 265), (...",270
ETV Plus India,"[(etv Plus, 253), (etv comedy show, 253), (3 I...",253
SAB TV,"[(sab tv serials, 214), (sab tv channel, 212),...",244
RadaanMedia,"[(Vaani rani, 242), (rani, 242), (Radhika in f...",243
mallemalatv,"[(All Time Hit Comedy Shows, 228), (Comedy Vid...",238
SET India,"[(the kapil sharma show, 158), (kapil sharma, ...",238
ETV Jabardasth,"[(etv shows, 234), (eenadu television, 234), (...",234


## 2. What are the top 25 tags in each country ?

- Approach 1 [ok]: Get the data and reproduce our logic
- Approach 2 [better]: We use a 
- Approach 3 [best]: We reuse our unique_tags function

In [21]:
# 1. Redo the logic
all_tags = df['tags'].apply(lambda x: x.split('|')).values
all_tags = [elem.strip('"') for row in all_tags for elem in row]
top25 = Counter(all_tags).most_common(25)
top25

[('comedy', 2384),
 ('funny', 2129),
 ('television', 1414),
 ('show', 1400),
 ('[none]', 1381),
 ('serial', 1330),
 ('full episode', 1303),
 ('funny videos', 1249),
 ('daily soap', 1235),
 ('watch online', 1229),
 ('latest news', 994),
 ('2018', 958),
 ('news', 919),
 ('jabardasth', 898),
 ('India', 836),
 ('bollywood', 820),
 ('entertainment', 806),
 ('telugu', 805),
 ('punjabi songs', 786),
 ('extra jabardasth', 783),
 ('latest', 772),
 ('india', 772),
 ('Zee5', 766),
 ('hindi', 765),
 ('breaking news', 697)]

In [22]:
# 2. Groupby obsession
# hackish, but works
def unique_tags(x, top: int=10):
    # split by '|'
    res = x.apply(lambda x: x.split('|')) 
    # flatten in single list
    flat = [elem.strip('"') for row in res for elem in row]
    # return a set (so it keeps only unique elements)
    return Counter(flat).most_common(top)

df['all'] = 1
df.groupby('all')[['tags']].agg(unique_tags, top=25)['tags'].values[0]

[('comedy', 2384),
 ('funny', 2129),
 ('television', 1414),
 ('show', 1400),
 ('[none]', 1381),
 ('serial', 1330),
 ('full episode', 1303),
 ('funny videos', 1249),
 ('daily soap', 1235),
 ('watch online', 1229),
 ('latest news', 994),
 ('2018', 958),
 ('news', 919),
 ('jabardasth', 898),
 ('India', 836),
 ('bollywood', 820),
 ('entertainment', 806),
 ('telugu', 805),
 ('punjabi songs', 786),
 ('extra jabardasth', 783),
 ('latest', 772),
 ('india', 772),
 ('Zee5', 766),
 ('hindi', 765),
 ('breaking news', 697)]

In [23]:
# 3. or simply this
unique_tags(df['tags'], top=25)

[('comedy', 2384),
 ('funny', 2129),
 ('television', 1414),
 ('show', 1400),
 ('[none]', 1381),
 ('serial', 1330),
 ('full episode', 1303),
 ('funny videos', 1249),
 ('daily soap', 1235),
 ('watch online', 1229),
 ('latest news', 994),
 ('2018', 958),
 ('news', 919),
 ('jabardasth', 898),
 ('India', 836),
 ('bollywood', 820),
 ('entertainment', 806),
 ('telugu', 805),
 ('punjabi songs', 786),
 ('extra jabardasth', 783),
 ('latest', 772),
 ('india', 772),
 ('Zee5', 766),
 ('hindi', 765),
 ('breaking news', 697)]

## Fix the bugs

- Remove the ugly [none]
- fix the casing